In [86]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import time
import pytz
import shapely
from shapely.geometry import Point, Polygon
from shapely import LineString
from shapely.wkt import loads
from datetime import datetime, timedelta
from pyproj import Proj
import pyproj
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import xml.etree.ElementTree as ET
import dask_geopandas as dgpd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from function import *
import os, datetime, glob, re, json, ast, requests

## XML Parsing

In [87]:
ROOT_PATH = r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data"
path = ROOT_PATH + r"\Raw\地震\地震目錄-歷史地震目錄\*"
[x.split("地震目錄-歷史地震目錄\\")[-1] for x in sorted(glob.glob(path))]

['CWA-EQ-Catalog-1990.xml',
 'CWA-EQ-Catalog-1991.xml',
 'CWA-EQ-Catalog-1992.xml',
 'CWA-EQ-Catalog-1993.xml',
 'CWA-EQ-Catalog-1994.xml',
 'CWA-EQ-Catalog-1995.xml',
 'CWA-EQ-Catalog-1996.xml',
 'CWA-EQ-Catalog-1997.xml',
 'CWA-EQ-Catalog-1998.xml',
 'CWA-EQ-Catalog-1999.xml',
 'CWA-EQ-Catalog-2000.xml',
 'CWA-EQ-Catalog-2001.xml',
 'CWA-EQ-Catalog-2002.xml',
 'CWA-EQ-Catalog-2003.xml',
 'CWA-EQ-Catalog-2004.xml',
 'CWA-EQ-Catalog-2005.xml',
 'CWA-EQ-Catalog-2006.xml',
 'CWA-EQ-Catalog-2007.xml',
 'CWA-EQ-Catalog-2008.xml',
 'CWA-EQ-Catalog-2009.xml',
 'CWA-EQ-Catalog-2010.xml',
 'CWA-EQ-Catalog-2011.xml',
 'CWA-EQ-Catalog-2012.xml',
 'CWA-EQ-Catalog-2013.xml',
 'CWA-EQ-Catalog-2014.xml',
 'CWA-EQ-Catalog-2015.xml',
 'CWA-EQ-Catalog-2016.xml',
 'CWA-EQ-Catalog-2017.xml',
 'CWA-EQ-Catalog-2018.xml',
 'CWA-EQ-Catalog-2019.xml',
 'CWA-EQ-Catalog-2020.xml',
 'CWA-EQ-Catalog-2021.xml',
 'CWA-EQ-Catalog-2022.xml',
 'CWA-EQ-Catalog-2023.xml',
 'file.csv',
 'manifest.csv',
 'schema-file.csv'

In [88]:
"""
由於歷年地震資料內筆數不一，觀察後發現有些會少rms~erh等欄位，但Quality、Reviewstatus的值都會有。
為了資料處理方便，將缺失的欄位依照資料筆數，從erz開始向前補上None。
中間可能會有缺失erh卻將erz的值登記到erh的狀況，但由於erh的缺失值較少，因此不做處理。
"""
# Path & Year Setting
year = [2018, 2019, 2020, 2021, 2022, 2023]
df = pd.DataFrame()

for year in year:
    xml_path = ROOT_PATH + r"\Raw\地震\地震目錄-歷史地震目錄\CWA-EQ-Catalog-" + f"{year}" + ".xml"
    # Read XML
    tree = ET.parse(xml_path)
    root = tree.getroot()
    df_list = []
    for i in range(2, len(root[8][1])):
        data = root[8][1][i]
        if len(data) == 14:
            temp = {
                "發震時間(OriginTime)": data[0].text,
                "震央經度(EpicenterLng)": float(data[1].text), 
                "震央緯度(EpicenterLat)": float(data[2].text),
                "震源深度(FocalDepth)": data[3].text,
                "芮氏規模(LocalMagnitude)": data[4].text,
                "定位測站個數(StationNumber)": data[5].text,
                "定位相位個數(PhaseNumber)": data[6].text,
                "震央距(MinimumEpicenterDistance)": data[7].text,
                "最大空餘角度(gap)": data[8].text,
                "震波走時殘差(rms)": data[9].text, 
                "水平標準偏差(erh)": data[10].text,
                "垂直標準偏差(erz)": data[11].text if len(data) > 13 else None,
                "定位品質(Quality)": data[12].text if len(data) > 13 else data[11].text,
                "定位回顧狀態(ReviewStatus)": data[13].text if len(data) > 13 else data[12].text,
            }
            df_list.append(temp)
        elif len(data) == 13:
            temp = {
                "發震時間(OriginTime)": data[0].text,
                "震央經度(EpicenterLng)": float(data[1].text), 
                "震央緯度(EpicenterLat)": float(data[2].text),
                "震源深度(FocalDepth)": data[3].text,
                "芮氏規模(LocalMagnitude)": data[4].text,
                "定位測站個數(StationNumber)": data[5].text,
                "定位相位個數(PhaseNumber)": data[6].text,
                "震央距(MinimumEpicenterDistance)": data[7].text,
                "最大空餘角度(gap)": data[8].text,
                "震波走時殘差(rms)": data[9].text, 
                "水平標準偏差(erh)": data[10].text,
                "垂直標準偏差(erz)": None,
                "定位品質(Quality)": data[11].text,
                "定位回顧狀態(ReviewStatus)": data[12].text,
            }
            df_list.append(temp)
        elif len(data) == 12:
            temp = {
                "發震時間(OriginTime)": data[0].text,
                "震央經度(EpicenterLng)": float(data[1].text), 
                "震央緯度(EpicenterLat)": float(data[2].text),
                "震源深度(FocalDepth)": data[3].text,
                "芮氏規模(LocalMagnitude)": data[4].text,
                "定位測站個數(StationNumber)": data[5].text,
                "定位相位個數(PhaseNumber)": data[6].text,
                "震央距(MinimumEpicenterDistance)": data[7].text,
                "最大空餘角度(gap)": data[8].text,
                "震波走時殘差(rms)": data[9].text, 
                "水平標準偏差(erh)": None,
                "垂直標準偏差(erz)": None,
                "定位品質(Quality)": data[10].text,
                "定位回顧狀態(ReviewStatus)": data[11].text
            }
            df_list.append(temp)
        elif len(data) < 12:
            temp = {
                "發震時間(OriginTime)": data[0].text,
                "震央經度(EpicenterLng)": float(data[1].text), 
                "震央緯度(EpicenterLat)": float(data[2].text),
                "震源深度(FocalDepth)": data[3].text,
                "芮氏規模(LocalMagnitude)": data[4].text,
                "定位測站個數(StationNumber)": data[5].text,
                "定位相位個數(PhaseNumber)": data[6].text,
                "震央距(MinimumEpicenterDistance)": data[7].text,
                "最大空餘角度(gap)": data[8].text,
                "震波走時殘差(rms)": None, 
                "水平標準偏差(erh)": None,
                "垂直標準偏差(erz)": None,
                "定位品質(Quality)": data[9].text,
                "定位回顧狀態(ReviewStatus)": data[10].text
            }
            df_list.append(temp)
        df_year = pd.DataFrame(df_list)
    df = pd.concat([df, df_year], axis=0)
    
df.shape    #35.8s

(13922, 14)

In [89]:
# year = 2018
# xml_path = ROOT_PATH + r"\地震\地震目錄-歷史地震目錄\CWA-EQ-Catalog-" + f"{year}" + ".xml"
# tree = ET.parse(xml_path)
# root = tree.getroot()
# print_xml_tree_without_namespace(root)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13922 entries, 0 to 2546
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   發震時間(OriginTime)               13922 non-null  object 
 1   震央經度(EpicenterLng)             13922 non-null  float64
 2   震央緯度(EpicenterLat)             13922 non-null  float64
 3   震源深度(FocalDepth)               13922 non-null  object 
 4   芮氏規模(LocalMagnitude)           13922 non-null  object 
 5   定位測站個數(StationNumber)          13922 non-null  object 
 6   定位相位個數(PhaseNumber)            13922 non-null  object 
 7   震央距(MinimumEpicenterDistance)  13922 non-null  object 
 8   最大空餘角度(gap)                    13922 non-null  object 
 9   震波走時殘差(rms)                    13921 non-null  object 
 10  水平標準偏差(erh)                    13920 non-null  object 
 11  垂直標準偏差(erz)                    13907 non-null  object 
 12  定位品質(Quality)                  13922 non-null  objec

In [92]:
df.to_excel(ROOT_PATH + r"\Processed\地震資料\cwa_earthquake_catalog_2018-2023.xlsx", index=False)
df.to_csv(ROOT_PATH + r"\Processed\地震資料\cwa_earthquake_catalog_2018-2023.csv", index=False)
print("Done")

Done


## 資料預覽與測試用Code

In [71]:
year = 2023
xml_path = ROOT_PATH + r"\地震\地震目錄-歷史地震目錄\CWA-EQ-Catalog-" + f"{year}" + ".xml"
# Read XML
tree = ET.parse(xml_path)
root = tree.getroot()

df_list = []
for i in range(2, len(root[8][1])):
    data = root[8][1][i]
    if len(data) == 14:
        temp = {
            "發震時間(OriginTime)": data[0].text,
            "震央位置(EpicenterLng,lat)": (float(data[1].text), float(data[2].text)),
            "震源深度(FocalDepth)": data[3].text,
            "芮氏規模(LocalMagnitude)": data[4].text,
            "定位測站個數(StationNumber)": data[5].text,
            "定位相位個數(PhaseNumber)": data[6].text,
            "震央距(MinimumEpicenterDistance)": data[7].text,
            "最大空餘角度(gap)": data[8].text,
            "震波走時殘差(rms)": data[9].text, 
            "水平標準偏差(erh)": data[10].text,
            "垂直標準偏差(erz)": data[11].text if len(data) > 13 else None,
            "定位品質(Quality)": data[12].text if len(data) > 13 else data[11].text,
            "定位回顧狀態(ReviewStatus)": data[13].text if len(data) > 13 else data[12].text,
        }
        df_list.append(temp)
    elif len(data) == 13:
        temp = {
            "發震時間(OriginTime)": data[0].text,
            "震央位置(EpicenterLng,lat)": (float(data[1].text), float(data[2].text)),
            "震源深度(FocalDepth)": data[3].text,
            "芮氏規模(LocalMagnitude)": data[4].text,
            "定位測站個數(StationNumber)": data[5].text,
            "定位相位個數(PhaseNumber)": data[6].text,
            "震央距(MinimumEpicenterDistance)": data[7].text,
            "最大空餘角度(gap)": data[8].text,
            "震波走時殘差(rms)": data[9].text, 
            "水平標準偏差(erh)": data[10].text,
            "垂直標準偏差(erz)": None,
            "定位品質(Quality)": data[11].text,
            "定位回顧狀態(ReviewStatus)": data[12].text,
        }
        df_list.append(temp)
    elif len(data) == 12:
        temp = {
            "發震時間(OriginTime)": data[0].text,
            "震央位置(EpicenterLng,lat)": (float(data[1].text), float(data[2].text)),
            "震源深度(FocalDepth)": data[3].text,
            "芮氏規模(LocalMagnitude)": data[4].text,
            "定位測站個數(StationNumber)": data[5].text,
            "定位相位個數(PhaseNumber)": data[6].text,
            "震央距(MinimumEpicenterDistance)": data[7].text,
            "最大空餘角度(gap)": data[8].text,
            "震波走時殘差(rms)": data[9].text, 
            "水平標準偏差(erh)": None,
            "垂直標準偏差(erz)": None,
            "定位品質(Quality)": data[10].text,
            "定位回顧狀態(ReviewStatus)": data[11].text
        }
        df_list.append(temp)
    elif len(data) < 12:
        temp = {
            "發震時間(OriginTime)": data[0].text,
            "震央位置(EpicenterLng,lat)": (float(data[1].text), float(data[2].text)),
            "震源深度(FocalDepth)": data[3].text,
            "芮氏規模(LocalMagnitude)": data[4].text,
            "定位測站個數(StationNumber)": data[5].text,
            "定位相位個數(PhaseNumber)": data[6].text,
            "震央距(MinimumEpicenterDistance)": data[7].text,
            "最大空餘角度(gap)": data[8].text,
            "震波走時殘差(rms)": None, 
            "水平標準偏差(erh)": None,
            "垂直標準偏差(erz)": None,
            "定位品質(Quality)": data[9].text,
            "定位回顧狀態(ReviewStatus)": data[10].text
        }
        df_list.append(temp)
df = pd.DataFrame(df_list)
df.head()
# 有些地震資訊只有erh無erz

,發震時間(OriginTime),"震央位置(EpicenterLng,lat)",震源深度(FocalDepth),芮氏規模(LocalMagnitude),定位測站個數(StationNumber),定位相位個數(PhaseNumber),震央距(MinimumEpicenterDistance),最大空餘角度(gap),震波走時殘差(rms),水平標準偏差(erh),垂直標準偏差(erz),定位品質(Quality),定位回顧狀態(ReviewStatus)
0,2023-01-01T09:30:05+08:00,"(122.414, 23.943)",11.3,3.1,10,14,22.00,240,0.18,1.3,0.8,D,F
1,2023-01-01T11:29:42+08:00,"(122.893, 24.385)",53.8,3.0,34,57,15.10,182,0.3,1.1,0.8,D,F
2,2023-01-01T12:33:54+08:00,"(121.162, 24.21)",59.9,3.2,99,236,4.30,17,0.31,0.2,0.2,B,F
3,2023-01-01T16:47:07+08:00,"(122.514, 24.014)",25.9,4.3,99,266,23.30,151,0.5,0.2,0.1,C,F
4,2023-01-01T22:29:53+08:00,"(121.996, 24.394)",5.5,3.2,83,141,22.80,83,0.34,0.4,0.4,C,F


In [41]:
len(df)

1828

In [66]:
count_list = []
for i in range(2, len(root[8][1])):
    data = root[8][1][i]
    temp = {
        "ID" : i,
        "長度": len(data)
    }
    count_list.append(temp)
count_df = pd.DataFrame(count_list)
count_df

,ID,長度
0,2,14
1,3,14
2,4,14
3,5,14
4,6,14
...,...,...
2542,2544,14
2543,2545,14
2544,2546,14
2545,2547,14


In [67]:
count_df["長度"].value_counts()

長度
14    2542
13       4
11       1
Name: count, dtype: int64

In [68]:
a = count_df["長度"] < 14
count_df[a]

,ID,長度
1050,1052,13
1312,1314,11
1636,1638,13
1994,1996,13
2317,2319,13


In [74]:
id = 1314
for a in range(len(root[8][1][id])):
    print(root[8][1][id][a].tag)

{urn:cwa:gov:tw:cwacommon:0.1}OriginTime
{urn:cwa:gov:tw:cwacommon:0.1}EpicenterLongitude
{urn:cwa:gov:tw:cwacommon:0.1}EpicenterLatitude
{urn:cwa:gov:tw:cwacommon:0.1}FocalDepth
{urn:cwa:gov:tw:cwacommon:0.1}LocalMagnitude
{urn:cwa:gov:tw:cwacommon:0.1}StationNumber
{urn:cwa:gov:tw:cwacommon:0.1}PhaseNumber
{urn:cwa:gov:tw:cwacommon:0.1}MinimumEpicenterDistance
{urn:cwa:gov:tw:cwacommon:0.1}gap
{urn:cwa:gov:tw:cwacommon:0.1}Quality
{urn:cwa:gov:tw:cwacommon:0.1}ReviewStatus


## Analysis

1. 爬取歷年來地震發生時間與地點位置
2. 發生時間約於天坑前5天，地點：(1)位於台北市內 (2)對台北市有影響者
3. 確認地震發生前後天坑數量

In [113]:
# Load Data
earthquake_raw = pd.read_excel(ROOT_PATH + r"\Processed\地震資料\cwa_earthquake_catalog_2018-2023.xlsx")
roadcase_raw = pd.read_excel(ROOT_PATH + r"\Processed\道管系統坑洞案件_108-112_Chu加案件標記_20240201.xlsx")
tp_shp_path = r"D:\Chu's Document!\03 Raw Data\SHP\臺北市區界圖_20220915\G97_A_CADIST_P.shp"
tp_shp_raw = gpd.read_file(tp_shp_path, encoding='utf-8', crs='EPSG:3826')

In [114]:
df_earthquake = earthquake_raw.copy()
df_roadcase = roadcase_raw.copy()
tp_shp = tp_shp_raw.copy()

### 篩選位於臺北市內地震

In [115]:
df_earthquake.head()

,發震時間(OriginTime),震央經度(EpicenterLng),震央緯度(EpicenterLat),震源深度(FocalDepth),芮氏規模(LocalMagnitude),定位測站個數(StationNumber),定位相位個數(PhaseNumber),震央距(MinimumEpicenterDistance),最大空餘角度(gap),震波走時殘差(rms),水平標準偏差(erh),垂直標準偏差(erz),定位品質(Quality),定位回顧狀態(ReviewStatus)
0,2018-01-01T11:21:57+08:00,121.241,22.865,8.9,3.9,99,176,12.3,74,0.33,0.1,0.1,C,F
1,2018-01-01T20:48:23+08:00,121.238,22.863,8.0,4.0,99,168,12.0,76,0.33,0.1,0.2,C,F
2,2018-01-02T03:22:03+08:00,121.239,22.863,8.8,4.3,99,198,12.1,74,0.35,0.1,0.2,C,F
3,2018-01-02T03:55:59+08:00,121.253,22.853,8.6,3.0,99,132,13.9,76,0.37,0.1,0.3,C,F
4,2018-01-02T04:10:28+08:00,121.247,22.851,7.6,3.8,99,190,13.5,77,0.33,0.1,0.2,C,F


In [116]:
gdf_earthquake = gpd.GeoDataFrame(df_earthquake, geometry=gpd.points_from_xy(df_earthquake["震央經度(EpicenterLng)"], df_earthquake["震央緯度(EpicenterLat)"]))
gdf_earthquake["發震時間(OriginTime)"] = pd.to_datetime(df_earthquake["發震時間(OriginTime)"])
gdf_earthquake.head()

,發震時間(OriginTime),震央經度(EpicenterLng),震央緯度(EpicenterLat),震源深度(FocalDepth),芮氏規模(LocalMagnitude),定位測站個數(StationNumber),定位相位個數(PhaseNumber),震央距(MinimumEpicenterDistance),最大空餘角度(gap),震波走時殘差(rms),水平標準偏差(erh),垂直標準偏差(erz),定位品質(Quality),定位回顧狀態(ReviewStatus),geometry
0,2018-01-01 11:21:57+08:00,121.241,22.865,8.9,3.9,99,176,12.3,74,0.33,0.1,0.1,C,F,POINT (121.24100 22.86500)
1,2018-01-01 20:48:23+08:00,121.238,22.863,8.0,4.0,99,168,12.0,76,0.33,0.1,0.2,C,F,POINT (121.23800 22.86300)
2,2018-01-02 03:22:03+08:00,121.239,22.863,8.8,4.3,99,198,12.1,74,0.35,0.1,0.2,C,F,POINT (121.23900 22.86300)
3,2018-01-02 03:55:59+08:00,121.253,22.853,8.6,3.0,99,132,13.9,76,0.37,0.1,0.3,C,F,POINT (121.25300 22.85300)
4,2018-01-02 04:10:28+08:00,121.247,22.851,7.6,3.8,99,190,13.5,77,0.33,0.1,0.2,C,F,POINT (121.24700 22.85100)


In [117]:
tp_shp.columns

Index(['TNAME', 'MAX_X', 'MAX_Y', 'MIN_X', 'MIN_Y', 'geometry'], dtype='object')

In [118]:
# Setting CRS
tp_shp.crs = "EPSG:3826"
tp_shp = tp_shp.to_crs(epsg=4326)
# gdf_earthquake.crs = "EPSG:4326"

# Spatial Join
tp_earthquake = gpd.sjoin(tp_shp, gdf_earthquake, how="inner", predicate="contains")

# Drop Columns
drop_col = ['MAX_X', 'MAX_Y', 'MIN_X', 'MIN_Y', 'geometry']
tp_earthquake = tp_earthquake.drop(columns=drop_col)
tp_earthquake.head()

C:\Users\doit\AppData\Local\Temp\ipykernel_12072\1507584487.py:7: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  tp_earthquake = gpd.sjoin(tp_shp, gdf_earthquake, how="inner", predicate="contains")


,TNAME,index_right,發震時間(OriginTime),震央經度(EpicenterLng),震央緯度(EpicenterLat),震源深度(FocalDepth),芮氏規模(LocalMagnitude),定位測站個數(StationNumber),定位相位個數(PhaseNumber),震央距(MinimumEpicenterDistance),最大空餘角度(gap),震波走時殘差(rms),水平標準偏差(erh),垂直標準偏差(erz),定位品質(Quality),定位回顧狀態(ReviewStatus)
0,北投區,2652,2019-08-07 01:00:42+08:00,121.553,25.155,2.6,3.4,43,71,1.2,50,0.30,0.2,0.2,B,F
0,北投區,2651,2019-08-07 01:00:40+08:00,121.553,25.165,4.6,3.1,98,144,0.9,45,0.21,0.1,0.1,B,F
0,北投區,11707,2023-02-17 09:08:20+08:00,121.559,25.162,4.7,3.4,99,173,1.4,42,0.26,0.1,0.1,B,F
1,士林區,1965,2019-02-10 04:12:57+08:00,121.541,25.133,7.7,4.0,99,209,3.2,29,0.34,0.2,0.1,B,F
1,士林區,13722,2023-12-02 17:52:11+08:00,121.566,25.115,134.6,3.2,68,112,3.6,69,0.26,1.2,0.6,A,F


In [119]:
tp_earthquake.shape

(16, 16)

In [121]:
tp_earthquake["發震時間(OriginTime)"] = tp_earthquake["發震時間(OriginTime)"].astype(str)
tp_earthquake.to_excel(ROOT_PATH + r"\Processed\地震資料\tp_earthquake_2018-2023.xlsx", index=False)